# Bank

In [ ]:
import sage
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bank()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_cols = ['Job', 'Marital', 'Education', 'Default', 'Housing',
                    'Loan', 'Contact', 'Month', 'Prev Outcome']
categorical_inds = [feature_names.index(col) for col in categorical_cols]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
with open('trained_models/bank model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
sage_final = sage.load('results/bank_sage.pkl')
final_values = sage_final.values

In [ ]:
# Setup and calculate
imputer = sage.MarginalImputer(model, test[:512])
estimator = sage.PermutationEstimator(imputer, 'cross entropy')

In [ ]:
values_list = []
for i in range(1024):
    sage_values = estimator(test, Y_test, batch_size=8, n_permutations=8, bar=False)
    values_list.append(sage_values.values)
    corr = np.corrcoef(np.array(values_list).mean(axis=0), final_values)[0, 1]
    print(i, corr)

In [ ]:
results_dict = {
    'values': np.array(values_list),
    'evals': 8 * len(feature_names),
    'inner_samples': 512
}
with open('results/bank sage convergence.pkl', 'wb') as f:
    pickle.dump(results_dict, f)

# Bike

In [ ]:
import sage
import pickle
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bike()
feature_names = df.columns.tolist()[:-3]

In [ ]:
# Split data, with total count serving as regression target
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy()
Y_val = val[:, -1].copy()
Y_test = test[:, -1].copy()
train = train[:, :-3].copy()
val = val[:, :-3].copy()
test = test[:, :-3].copy()

In [ ]:
with open('trained_models/bike model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
sage_final = sage.load('results/bike_sage.pkl')
final_values = sage_final.values

In [ ]:
# Setup and calculate
imputer = sage.MarginalImputer(model, test[:512])
estimator = sage.PermutationEstimator(imputer, 'mse')

In [ ]:
values_list = []
for i in range(512):
    sage_values = estimator(test, Y_test, batch_size=1, n_permutations=1, bar=False)
    values_list.append(sage_values.values)
    corr = np.corrcoef(np.array(values_list).mean(axis=0), final_values)[0, 1]
    print(i, corr)

In [ ]:
results_dict = {
    'values': np.array(values_list),
    'evals': 1 * len(feature_names),
    'inner_samples': 512
}
with open('results/bike sage convergence.pkl', 'wb') as f:
    pickle.dump(results_dict, f)

# Credit

In [ ]:
import sage
import numpy as np
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.credit()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_columns = [
    'Checking Status', 'Credit History', 'Purpose', 'Credit Amount',
    'Savings Account/Bonds', 'Employment Since', 'Personal Status',
    'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
    'Housing Ownership', 'Job', 'Telephone', 'Foreign Worker'
]
categorical_inds = [feature_names.index(col) for col in categorical_columns]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
with open('trained_models/credit model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
sage_final = sage.load('results/credit_sage.pkl')
final_values = sage_final.values

In [ ]:
# Setup and calculate
imputer = sage.MarginalImputer(model, train[:512])
estimator = sage.PermutationEstimator(imputer, 'cross entropy')

In [ ]:
values_list = []
for i in range(1024):
    sage_values = estimator(test, Y_test, batch_size=1, n_permutations=1, bar=False)
    values_list.append(sage_values.values)
    corr = np.corrcoef(np.array(values_list).mean(axis=0), final_values)[0, 1]
    print(i, corr)

In [ ]:
results_dict = {
    'values': np.array(values_list),
    'evals': 1 * len(feature_names),
    'inner_samples': 512
}
with open('results/credit sage convergence.pkl', 'wb') as f:
    pickle.dump(results_dict, f)